# BigEyes - The only surveillance system you'll need

### Install dependencies

In [ ]:
!pip install opencv-python face_recognition requests plyer pyobjus playsound

### Import packages and assign variables

In [ ]:
import json
import cv2
import face_recognition
import requests
from os import listdir, system
from os.path import isdir
from sys import platform
from re import match
from plyer import notification
from playsound import playsound
import matplotlib.pyplot as plt
from datetime import datetime

with open('config.json') as f: config = json.load(f)

known_encodings = []

for file in listdir('authorized_users'):
    known_encodings.append(
        face_recognition.face_encodings(face_recognition.load_image_file('authorized_users/' + file))[0]
    )

### Add new user function

In [ ]:
def add_new_authorized_user():
    name = input('Hi! Enter a name to add a new authorized user. This name will be used to identify the user and cannot be changed or used for another user.\n')
    if '.' in name or '/' in name:
        return print('Names can\'t have dots or forward slashes! Please try again.')
    for authorized_user in listdir('authorized_users'):
        if authorized_user.split('.')[0].lower() == name.lower():
            return print(f'This name is already in use! Please choose another name or delete the user {name}.')
    
    image = input(f'Alright, {name} it is. Enter a path to an image of the face of the user you want to add. Make sure this image is clear and that the user is directly facing the camera.\n')

    print('Processing the image...')
    image = cv2.imread(image)
    if image is None:
        return print('That doesn\'t seem to be a valid image file path! Please try again.')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    faces = face_recognition.face_locations(image)
    face_count = len(faces)
    if face_count == 0:
        return print('No faces found in the image! Please try again.')
    elif face_count > 1:
        print('More than one face found in the image! Please try again.')
        # draw rectangle around each face
        for (top, right, bottom, left) in faces:
            cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 2)
        plt.imshow(image)
        plt.axis('off')
        return plt.show()
    else:
        cv2.imwrite(f'authorized_users/{name}.jpg', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        print('Image saved!')
        (top, right, bottom, left) = faces[0]
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        plt.imshow(image)
        plt.axis('off')
        plt.show()

# add_new_authorized_user()

### Configuration function

In [ ]:
def setup_user_config():
    global config
    
    flag = True
    while flag:
        folder = input('I will be saving all the pictures of unauthorized intruders that wrongly entered the area in a folder of your choice. Create a folder for this and enter the path of this folder here.\n')
        if not isdir(folder):
            print('That doesn\'t seem to be a valid folder path! Please try again.')
        else:
            if folder.endswith('/'):
                folder = folder[:-1]
            print('OK, folder set. Make sure to check this folder regularly for the pictures of unauthorized intruders.')
            flag = False

    with open('config.json') as f: config = json.load(f)
    config['folder'] = folder
    with open('config.json', 'w') as f: json.dump(config, f, indent = 4)

    flag = True
    while flag:
        announce = input('Should I announce a warning to the intruder that entered the area verbally? This could warn them off the area. The alert would still be sent to your folder (yes/no).\n').lower()
        if announce == 'yes' or announce == 'y':
            print('OK, I will announce the warning verbally to the intruder.')
            flag = False
        elif announce == 'no' or announce == 'n':
            print('OK, I will not announce the warning verbally to the intruder.')
            flag = False
        else:
            print('That doesn\'t seem to be a valid answer! Try answering yes or no.')

    with open('config.json') as f: config = json.load(f)
    config['announce'] = True if announce == 'yes' or announce == 'y' else False
    with open('config.json', 'w') as f: json.dump(config, f, indent = 4)

    flag = True
    while flag:
        try:    
            tolerance = (float(
                input('How strict should the face recognition be? This is the tolerance level for how similar the face in the camera should be to the known faces. The recommended is 6, minimum is 1 and maximum is 10. The lower the number, the stricter recognition will be. ' \
                    'If you\'re unsure/not facing issues, type 6. If the camera is recognizing unauthorized people as authorized people, reduce the number. If it is doing the opposite, increase the number.' \
                    f' Your current tolerance level is {config["tolerance"]}.' if config.get('tolerance') else ''
                )
            )) / 10
            if tolerance < 0.1 or tolerance > 1:
                raise ValueError
            flag = False
        except ValueError:
            print('That doesn\'t seem to be a valid number! It must be a number which is greater than 1 and less than 10. Please try again.')

    with open('config.json') as f: config = json.load(f)
    config['tolerance'] = tolerance
    with open('config.json', 'w') as f: json.dump(config, f, indent = 4)

    print('Configuration done!')

# setup_user_config()

### Webhook setup function

In [ ]:
def setup_webhook():
    print('Tip: You should read the webhook guide using "webhook help".')
    
    global config
    
    flag = True
    while flag:
        webhook = input('Enter the webhook URL of the webhook you want to use to send the alert to.\n')
        if not match(r'^https?:\/\/.*', webhook):
            print('That doesn\'t seem to be a valid URL! Please try again.')
        else:
            print('OK, I\'ll send a test request on the webhook...')
            try:
                req = requests.post(webhook, json = {'content': 'Test message'})
                if not 299 >= req.status_code >= 200:
                    return print(f'Hm, the webhook responded with status code {req.status_code}! Please try again, it needs to respond with a status code between 200 - 299.')
            except requests.ConnectionError:
                return print('Could not connect to the webhook! Please try again and/or make sure your URL is valid.')
            print('OK, webhook set! Whenever an unauthorized user enters the area, I will send a POST request to the webhook.')
            flag = False

    with open('config.json') as f: config = json.load(f)
    config['webhook'] = webhook
    with open('config.json', 'w') as f: json.dump(config, f, indent = 4)

### Webhook help function

In [ ]:
def webhook_help():
    print('''
This section is targeted mainly at developers.
You can setup 1 webhook using the "webhook" command.
Read here to know what a webhook is - https://www.getvero.com/resources/webhooks/ - In this case, it's a URL that will receive a POST request whenever an unauthorized user enters the area
Read here to know more about POST requests - https://apipheny.io/what-is-post-request/
Whenever an unauthorized user enters the camera's field of view, the webhook will be sent a POST request with the following type of JSON data:
{
    "datetime": "DD-MM-YYYY at HH-MM-SS",
    "imagepath": "/YOUR/ALERTS/FOLDER/PATH/DD-MM-YYYY at HH-MM-SS.jpg",
}
Here, "DD-MM-YYYY at HH-MM-SS" is the format of date and time when the alert was sent (24-hr format).
"/YOUR/ALERTS/FOLDER/PATH/" is the alerts folder you set during the initial configuration, where the pictures of unauthorized users will be saved.

Upon receiving a POST request from BigEyes, the developer will recieve the JSON body above for details in their webhook and can take any action based on it using their own coding.
Eg. Setting up your own website as the webhook, coding your website to take its own customized action on the unauthorized intruder.

In essence, with a little web development and coding skills, you can setup your own website to take whatever action you want to take whenever an unauthorized intruder enters! Whether that be triggering an internet controlled alarm system, sending the intruder data somewhere else, etc. The power is in your hands.
    ''')

webhook_help()

### Enable camera detection

In [ ]:
def enable_camera():
    cascade_path = 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    
    print('Video started!')
    video_capture = cv2.VideoCapture(0)
    # loop over frames from the video file stream
    previous_frame_had_unauthorized_entrance = False
    while True:
        # grab the frame from the threaded video stream
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (60, 60),
            flags = cv2.CASCADE_SCALE_IMAGE
        )

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(rgb)
        
        if len(encodings) == 0:
            previous_frame_had_unauthorized_entrance = False

        for encoding in encodings:
            matches = face_recognition.compare_faces(
                known_encodings,
                encoding,
                tolerance = config.get('tolerance') or 0.6
            )
            face_locations = face_recognition.face_locations(rgb)
            unauthorized_entrance = True
            for match in matches:
                if bool(match): # match is a numpy bool so have to convert it to normal bool
                    unauthorized_entrance = False
                    break

            if unauthorized_entrance:
                print('Unauthorized entrance!')
                if not previous_frame_had_unauthorized_entrance:
                    for (top, right, bottom, left) in face_locations:
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    
                    time_now = datetime.now().strftime("%d-%m-%Y at %H-%M-%S")
                    cv2.imwrite(f'{config["folder"]}/{time_now}.jpg', frame)
                
                    try:
                        notification.notify(
                            title = 'BigEyes - Intruder alert!',
                            message = f'An intruder has been detected in the camera\'s area! See their picture in your alerts folder - {config["folder"]}.',
                            timeout = 100
                        )
                    except:
                        if platform == 'darwin': # mac support, run native AppleScript notification code
                            system(
                                f'osascript -e \'display notification "An intruder has been detected in the camera area! See their picture in your alerts folder - {config["folder"]}." with title "BigEyes - Intruder alert!"\''
                            )

                    if config.get('webhook'):
                        requests.post(config['webhook'],
                            json = {
                                "datetime": time_now,
                                "imagepath": config['folder'] + f'/{time_now}.jpg'
                            }
                        )

                if config['announce']:
                    playsound('unauthorized.mp3')

                previous_frame_had_unauthorized_entrance = True
            
            else:
                previous_frame_had_unauthorized_entrance = False

# enable_camera()

### Help function

In [ ]:
def help_func():
    print('BigEyes is a program that detects faces in a video stream and alerts you if an intruder enters the area.\n')
    print('Commands:\n')
    print('"New" - Adds a new authorized user for the surveillance system.')
    print('"Config" - Sets up some configuration like alert system folder path, whether to announce the alert to the intruder, tolerance etc. You need to setup this config before you can activate the camera.')
    print('"Webhook" - (Optional) Setup a webhook for when an unauthorized person enters the area. A webhook is a URL to which a "POST" type web request is sent. This is targeted at developers. Type "webhook help" to see more details.')
    print('"Webhook help" - Shows more details on how and why to setup a webhook.')
    print('"Activate" - Enables the camera and starts the video stream! Your security guard will be online.')
    print('"Help" - Shows this help message.')
    print('"Exit" - Stops BigEyes, or you can stop this cell.')
    print('Tip: Activate the camera when your device is placed inside the area you want to guard in front of the entrance for the best experience.')

### Main cell to execute

In [ ]:
print('"BigEyes - Your second pair of eyes" is now running!\n--------------------------------')
print('Type "help" to see what you can do with me.')

try:
    while True:
        command = input('> ').lower().strip()
        if command == 'new':
            add_new_authorized_user()
        elif command == 'config':
            setup_user_config()
        elif command == 'webhook':
            setup_webhook()
        elif command == 'webhook help':
            webhook_help()
        elif command == 'activate':
            if config == {}:
                print('You need to setup the configuration before adding a new authorized user! Type "config" to do that.')
            else:
                enable_camera()
        elif command == 'help':
            help_func()
        elif command == 'exit' or command == 'bye':
            raise KeyboardInterrupt
        else:
            print('That doesn\'t seem to be a valid command! Type "help" to see what you can do with me.')
except KeyboardInterrupt:
    print('\n--------------------------------')
    print('"BigEyes - Your second pair of eyes" is now stopping :(\nHope to see you again soon.')
    print('--------------------------------')